In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', sep='\t', error_bad_lines=False, header=None)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [4]:
df1 = pd.read_csv('./lastfm-dataset-1K/userid-profile.tsv', sep='\t', error_bad_lines=False, header=None)

In [12]:
len(df)

19098853

In [7]:
df.head(400)

,0,1,2,3,4,5
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Improvisation (Live_2009_4_15)
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Glacier (Live_2009_4_15)
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Parolibre (Live_2009_4_15)
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Bibo No Aozora (Live_2009_4_15)


In [11]:
df.isna().sum()

0          0
1          0
2     600848
3          0
4    2162719
5         12
dtype: int64

In [9]:
len(df1)

993

In [14]:
df.head()

,0,1,2,3,4,5
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [15]:
df.rename(columns={0: 'user_id', 1: 'timestamp', 2: 'abc', 3: 'artist', 4: 'album', 5: 'song'}, inplace=True)

In [20]:
df.drop(columns=['abc', 'album'], inplace=True)

In [97]:
test = df.sample(n=100000)

In [98]:
import numpy as np

In [99]:
test2 = pd.pivot_table(test, values='artist', index=['user_id', 'song', 'artist'], aggfunc=np.count_nonzero)

In [100]:
test3 = pd.DataFrame(test2)

In [101]:
test4 = test3.reset_index()

In [102]:
test4.set_index('user_id', inplace=True)

In [103]:
test4.rename(columns={'timestamp': 'playcount'}, inplace=True)

In [104]:
total_plays_df = test4.groupby('user_id').sum()

In [105]:
total_plays_df.rename(columns={'playcount': 'user_total_plays'}, inplace=True)

In [106]:
song_df = test4.join(total_plays_df)

In [107]:
song_df['score'] = song_df.playcount / song_df.user_total_plays

In [108]:
song_df

,song,artist,playcount,user_total_plays,score
user_id,,,,,
user_000001,Allright,Onra & Quetzal,2,98,0.020408
user_000001,"Amazing Featuring Tony Momrelle, Imaani (Taran...",Reel People,1,98,0.010204
user_000001,Are We? (Feat. Michelle Amador),Karma,1,98,0.010204
user_000001,Around The World,Christian Prommer'S Drumlesson,1,98,0.010204
user_000001,Backspin,Marc Mac,1,98,0.010204
user_000001,Beep It,Cornelius,1,98,0.010204
user_000001,Beginners Falafel,Flying Lotus,1,98,0.010204
user_000001,Beyond Skin,Nitin Sawhney,1,98,0.010204
user_000001,Botanica,高木正勝,1,98,0.010204


In [118]:
song_artist_list = list(zip(song_df.song, song_df.artist))

In [120]:
song_artist_list = list(set(song_artist_list))

In [199]:
len(song_artist_list)

72774

#### Spotify API

In [125]:
import requests
client_id = '8787e23c39884c2b98026feff5b7acde'
client_secret = '94d8c07f39f841739a58e5e12c9e5e86'

grant_type = 'client_credentials'

#Request based on Client Credentials Flow from https://developer.spotify.com/web-api/authorization-guide/

#Request body parameter: grant_type Value: Required. Set it to client_credentials
body_params = {'grant_type' : grant_type}

url='https://accounts.spotify.com/api/token'

response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.text)

{"access_token":"BQDR7bQAzfn1dF2SwtF67-yY4IdDKVWMm_Oy8DP-a9QRKdwI4nGfNBmyI880GVQpE7LbQkrHObxrQIejXnU","token_type":"Bearer","expires_in":3600,"scope":""}


In [126]:
access_token = 'BQDR7bQAzfn1dF2SwtF67-yY4IdDKVWMm_Oy8DP-a9QRKdwI4nGfNBmyI880GVQpE7LbQkrHObxrQIejXnU'
headers = {'Authorization': 'Bearer '+ access_token}

In [180]:
r = requests.get('https://api.spotify.com/v1/search?q=artist:Kendrick%20lamar%20track:humble&type=track', headers=headers)

In [181]:
data = r.json()


In [182]:
data['tracks']['items'][0]['id']

'7KXjTSCq5nL1LoYtL7XAwS'

In [200]:
from tqdm import tqdm
def get_track_ids(tracklist):
    id_list = {}
    for track in tqdm(tracklist):
        artist = track[1].replace(" ", "%20")
        song = track[0].replace(" ", "%20")
        try:
            r = requests.get(f'https://api.spotify.com/v1/search?q=artist:{artist}%20track:{song}&type=track', headers=headers)
            data = r.json()
            track_id = data['tracks']['items'][0]['id']
            id_list[track] = track_id
        except:
            id_list[track] = "NA"
    return id_list

In [201]:
ids = get_track_ids(song_artist_list)

100%|██████████| 72774/72774 [2:06:37<00:00,  9.58it/s]   


#### getting audio features from spotify API